In [130]:
is_stack = [True, False][0]
debug = False
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
import time
import datetime
import glob
sys.path.append('../py/')
from s027_kfold_ods import ods_kfold
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import logger_func, get_categorical_features, get_numeric_features, reduce_mem_usage, elo_save_feature, impute_feature
try:
    if not logger:
        logger=logger_func()
except NameError:
    logger=logger_func()

from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
from sklearn.linear_model import Ridge
#========================================================================


#========================================================================
# Args
out_part = ['', 'part', 'all'][0]
key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
stack_name='ridge'
submit = pd.read_csv('../input/sample_submission.csv')
model_type='ridge'
start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())
seed = 328
#========================================================================


#========================================================================
# Data Load 
print("Preparing dataset...")

if is_stack:
    is_clf_out = [True, False][1]
    is_no_out_flg = [True, False][1]
    is_rm_out = [True, False][1]
    
    base = utils.read_df_pkl('../input/base_no_out_clf.gz').set_index(key)
    
    #========================================================================
    # Base Model Path
    #========================================================================
    # Clf Out Model
    if is_clf_out:
        ens_list = glob.glob('../ensemble/clf_min_thres_ensemble/*.gz')
    # No Out Flg Model
    elif is_no_out_flg:
        ens_list = glob.glob('../no_out_flg_ensemble/*.gz')
    #========================================================================
    elif is_rm_out:
        ens_list = glob.glob('../ensemble/rm_outlier_ensemble/*.gz')
    #========================================================================
    else:
        lgb_list = glob.glob('../ensemble/lgb_ensemble/*.gz')
#         nn_list = glob.glob('../ensemble/NN_ensemble/*.gz')
        nn_list = []
        ens_list = lgb_list + nn_list
    
    #========================================================================
    # Stack Models Load
    from joblib import Parallel, delayed
    def parallel_stack_model(model_path):
        try:
            cv = re.search(r'CV([^/.]*)_LB.gz', model_path).group(1)
        except AttributeError:
            cv = re.search(r'CV([^/.]*).gz', model_path.replace('.', '-')).group(1)
        tmp = utils.read_pkl_gzip(model_path)
        if 'pred_mean' in tmp.columns:
            tmp = tmp[[key, 'pred_mean']]
        else:
            tmp = tmp[[key, 'prediction']]
            
        if model_path.count('lgb'):
            tmp.columns = [key, f"base_lgb_{cv}"]
        elif model_path.count('NN'):
            tmp.columns = [key, f"base_NN_{cv}"]
        else:
            tmp.columns = [key, f"base_model_{cv}"]
        return tmp.set_index(key)
    #========================================================================
    
    p_list = Parallel(n_jobs=-1)([delayed(parallel_stack_model)(model_path) for model_path in ens_list])
    df_pred = pd.concat(p_list, axis=1)
    base = base.join(df_pred)
    
    #========================================================================
    
    #========================================================================
    # Classifier
#     clf = utils.read_pkl_gzip('../stack/0207_224_outlier_classify_9seed_lgb_binary_CV0-9099420278047783_235features.gz')[[key, 'pred_mean']].set_index(key)
#     clf_2 = utils.read_pkl_gzip('../stack/0207_212_outlier_classify_9seed_lgb_binary_CV0-9084737642836664_235features.gz')[[key, 'pred_mean']].set_index(key)
#     clf['pred_mean_2'] = clf_2['pred_mean']
#     clf['clf_pred'] =  clf['pred_mean'].values*0.9 + clf['pred_mean_2'].values*0.1
    # clf['clf_pred'] =  clf['pred_mean']
    #========================================================================
    
    #========================================================================
    # Indexをそろえる
#     base['clf_pred'] = clf['clf_pred']
#     base_cols = [col for col in base.columns if col.count('base_')]
    # base = base[[target, 'clf_pred', 'no_out_flg'] + base_cols + out_cols + no_out_cols]
    #========================================================================
    
    if key in base.columns:
        train = base[~base[target].isnull()]
        test = base[base[target].isnull()]
    else:
        train = base[~base[target].isnull()].reset_index()
        test = base[base[target].isnull()].reset_index()
    
    if is_rm_out:
        train = train[~train[target].isnull()]
    elif is_clf_out:
        train = train[train['clf_pred']<0.01]
        test = test[test['clf_pred']<0.01]
        
    display(train.head())
    
    
else:
    win_path = f'../features/4_winner/*.gz'
    # Ensemble 1
    win_path = f'../model/LB3670_70leaves_colsam0322/*.gz'
    # Ensemble 2
    # win_path = f'../model/E2_lift_set/*.gz'
    # Ensemble 3
    # win_path = f'../model/E3_PCA_set/*.gz'
    
    win_path_list = glob.glob(win_path)
    
    base = utils.read_df_pkl('../input/base_term*0*')[[key, target, 'first_active_month']]
    base_train = base[~base[target].isnull()].reset_index(drop=True)
    base_test = base[base[target].isnull()].reset_index(drop=True)
    feature_list = utils.parallel_load_data(path_list=win_path_list)
    df = pd.concat(feature_list, axis=1)
    train = pd.concat([base_train, df.iloc[:len(base_train), :]], axis=1)
    test = pd.concat([base_test, df.iloc[len(base_train):, :].reset_index(drop=True)], axis=1)
    
    train.reset_index(inplace=True, drop=True)
    test.reset_index(inplace=True , drop=True)
#========================================================================

  0%|          | 0/1 [00:00<?, ?it/s]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Preparing dataset...


100%|██████████| 1/1 [00:00<00:00,  4.04it/s]


,card_id,target,first_active_month,hist_purchase_date_max,hist_purchase_month_max,hist_purchase_date_min,hist_purchase_month_min,new_purchase_date_max,new_purchase_month_max,new_purchase_date_min,new_purchase_month_min,hist_personal_term,new_personal_term,hist_regist_term,new_regist_term,no_out_flg,clf_pred,base_lgb_3-643008963792204,base_lgb_3-6335126722555198,base_lgb_3-6675809308648986,base_lgb_3-621674473874384,base_lgb_3-6213553126969393,base_lgb_3-674335359033703,base_lgb_3-6297296108420443,base_lgb_3-640200217753496,base_lgb_3-2381280632566187,base_lgb_3-240003171265297,base_lgb_3-6431931116870206,base_lgb_3-651455934282057,base_lgb_3-643556624877276,base_lgb_3-6246353407463157,base_lgb_3-6707091161311785,base_lgb_3-6812204136943496,base_lgb_3-6236254858483243,base_lgb_3-6333204401002663,base_lgb_3-627555_LB3-675,base_lgb_3-653674740088933,base_lgb_3-665786223163817
0,C_ID_92a2005557,-0.820283,2017-06-01,2018-02-25 09:31:15,2018-03-01,2017-06-27 14:18:08,2017-06-01,2018-04-29 11:23:05,2018-05-01,2018-03-05 14:04:36,2018-03-01,9,2.0,9,11.0,1.0,0.000444,-0.533682,-0.321566,-0.449476,-0.349492,-0.360447,-0.348939,-0.346031,-0.316781,-0.403515,-0.344339,-0.262261,-0.257492,-0.193392,-0.348467,-0.478001,-0.502368,-0.344292,-0.331135,-0.349346,-0.181378,-0.517733
1,C_ID_3d0044924f,0.392913,2017-01-01,2018-01-31 22:31:09,2018-02-01,2017-01-06 16:29:42,2017-01-01,2018-03-30 06:48:26,2018-04-01,2018-02-01 17:07:54,2018-02-01,13,2.0,13,15.0,0.0,0.007831,-0.669546,-0.581695,-0.971562,-0.598341,-0.199257,-0.493447,-0.557220,-0.308836,-1.136101,-0.955046,-0.586575,-0.052061,-0.180970,-0.746501,-0.847191,-0.892496,-0.349898,-0.484170,-0.627140,-0.652553,-1.128744
2,C_ID_d639edf6cd,0.688056,2016-08-01,2018-02-27 19:08:25,2018-03-01,2017-01-11 08:21:22,2017-01-01,2018-04-28 17:43:11,2018-05-01,2018-04-28 17:43:11,2018-04-01,14,1.0,19,18.0,0.0,0.004074,0.581285,0.849526,0.447654,0.655104,0.589762,0.801311,0.915316,0.586407,0.744735,0.651148,0.610568,0.759758,0.647221,0.677786,0.356035,0.454059,0.803631,0.642721,0.648794,0.584152,0.428851
3,C_ID_186d6a6901,0.142495,2017-09-01,2018-02-28 11:44:40,2018-03-01,2017-09-26 16:22:21,2017-09-01,2018-04-18 11:00:11,2018-05-01,2018-03-07 11:55:06,2018-03-01,6,2.0,6,8.0,0.0,0.000797,0.118323,0.214690,0.136579,0.147356,0.149292,0.027062,0.122011,0.143492,0.132069,0.012830,0.040901,0.000617,0.170643,0.153740,0.126440,0.189152,0.137893,0.108993,0.111918,0.205327,0.095453
4,C_ID_cdbd2c0db2,-0.159749,2017-11-01,2018-02-28 20:40:41,2018-03-01,2017-11-12 00:00:00,2017-11-01,2018-04-28 18:50:25,2018-05-01,2018-03-02 11:55:43,2018-03-01,4,2.0,4,6.0,1.0,0.000251,-0.282362,-0.137133,-0.216872,-0.079374,-0.105797,-0.200462,-0.223672,-0.296002,-0.126540,-0.218226,-0.071952,-0.180205,-0.003285,-0.078756,-0.303689,-0.308326,-0.120741,-0.106126,-0.182233,-0.205778,-0.489964


In [137]:
#========================================================================
# CVの準備
fold = 6
if is_rm_out:
    set_type = 'rm_out'
else:
    set_type = 'all'
# kfold = ods_kfold(train=train, seed=328, fold=fold)
# train.drop('rounded_target', axis=1, inplace=True)
if is_rm_out:
    kfold = utils.read_pkl_gzip('../input/kfold_ods_no_out_fold6_seed328.gz')
elif is_clf_out:
    kfold = utils.read_pkl_gzip('../input/kfold_ods_clf_out_fold6_seed328.gz')
else:
    kfold = utils.read_pkl_gzip('../input/kfold_ods_all_fold6_seed328.gz')

#========================================================================
# Dataset
submit = pd.read_csv('../input/sample_submission.csv').set_index(key)
model_list = []
result_list = []
score_list = []
val_pred_list = []
test_pred = np.zeros(len(test))

ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term', 'no_out_flg']
ignore_list = [key, target, 'merchant_id', 'first_active_month', 'index', 'personal_term']
Y = train[target]
y_mean = Y.mean()
#========================================================================
    
#========================================================================
# NN Model Setting 
fit_intercept = True
alpha = 0.4
# alpha = 1.0
max_iter = 1000
normalize = False
tol = 0.01
param_list = np.arange(1, 10, 1)
param_list = [0]
# param_list = np.arange(0.1, 1.0, 0.1)

#========================================================================
# 学習に使うカラム設定 
# base_lgb_cols = sorted([col for col in base.columns if col.count('base_lgb')])[2:20]
# use_cols = sorted([col for col in base.columns if col.count('base_')])
lgb_cols = sorted([col for col in base.columns if col.count('lgb_')])[:20]
nn_cols = []
other_cols = []
use_cols = lgb_cols + nn_cols + other_cols
# nn_cols = sorted([col for col in base.columns if col.count('NN_')])

# lgb_cols = list(set([col for col in train.columns if col not in ignore_list and col.count('lgb')]) - set(top3_lgb))
#========================================================================


seed_list = np.arange(100)
seed_list = [1]
best_score = 100
best_score_list = []
# stack_num = 6 - len(top3_lgb)

for seed in seed_list:

#     np.random.seed(seed)
#     use_cols = top3_lgb + list(np.random.choice(lgb_cols, size=stack_num, replace=False))
#     use_cols = lgb_cols[:10] + nn_cols[:3]
    
    result_list = []
    ridge = Ridge(solver='auto', fit_intercept=fit_intercept, alpha=alpha, max_iter=max_iter, normalize=normalize, tol=tol)
#     ridge = Ridge(solver='auto', fit_intercept=True, alpha=0.4, max_iter=200, normalize=False, tol=0.01)
    #========================================================================
    
    #========================================================================
    # Train & Prediction Start
    for fold_no, (trn_idx, val_idx) in enumerate(zip(*kfold)):
        if key not in train.columns:
            train = train.reset_index()
            test = test.reset_index() 
            
    
        #========================================================================
        # Make Dataset
        scaler = StandardScaler()
        scaler.fit(pd.concat([train[use_cols], test[use_cols]]))
        x_test = scaler.transform(test[use_cols])

        X_train, y_train = train.loc[train[key].isin(trn_idx), :][use_cols], Y.loc[train[key].isin(trn_idx)]
        X_val, y_val = train.loc[train[key].isin(val_idx), :][use_cols], Y.loc[train[key].isin(val_idx)]
        
        X_train[:] = scaler.transform(X_train)
        X_val[:] = scaler.transform(X_val)
        X_train = X_train.as_matrix()
        X_val = X_val.as_matrix()
    
        print(f"Train: {X_train.shape} | Test: {x_test.shape}")
        #========================================================================
        
        # Fitting
        ridge.fit(X_train, y_train)
        
        # Prediction
        y_pred = ridge.predict(X_val)
        test_pred += ridge.predict(x_test)
        
        # Stack Prediction
    #     df_pred = train.iloc[val_idx, :][[key, target]].copy()
        df_pred = train.loc[train[key].isin(val_idx), :][[key, target]].copy()
        df_pred['prediction'] = y_pred
        result_list.append(df_pred)
        
        # Scoring
        err = (y_val - y_pred)
        score = np.sqrt(mean_squared_error(y_val, y_pred))
        print(f'RMSE: {score} | SUM ERROR: {err.sum()}')
        score_list.append(score)
        #========================================================================
    
    cv_score = np.mean(score_list)
    
    #========================================================================
    # Stacking
    test_pred /= fold_no+1
    test['prediction'] = test_pred
    stack_test = test[[key, 'prediction']]
    result_list.append(stack_test)
    df_pred = pd.concat(result_list, axis=0, ignore_index=True).drop(target, axis=1)
    if key not in base:
        base.reset_index(inplace=True)
    df_pred = base[[key, target]].merge(df_pred, how='inner', on=key)
    print(f"Stacking Shape: {df_pred.shape}")
    #========================================================================
    
    #========================================================================
    # outlierに対するスコアを出す
    if is_rm_out:
        out_score = 0
    else:
        if key not in train.columns:
            train.reset_index(inplace=True)
        out_ids = train.loc[train.target<-30, key].values
        out_val = train.loc[train.target<-30, target].values
        out_pred = df_pred[df_pred[key].isin(out_ids)]['prediction'].values
        out_score = np.sqrt(mean_squared_error(out_val, out_pred))
    #========================================================================
    
    if cv_score<best_score:
        print(f'''
#========================================================================
# CV SCORE AVG: {cv_score}
# OUT SCORE: {out_score}
#========================================================================''')
    
        best_score = cv_score
        best_score_list = use_cols
    
        #========================================================================
        # Save Stack
        utils.to_pkl_gzip(path=f"../stack/{start_time[4:12]}_stack_{model_type}_set-{set_type}_lgb{len(lgb_cols)}_NN{len(nn_cols)}_other{len(other_cols)}_OUT{str(out_score)[:7]}_CV{cv_score}_LB" , obj=df_pred[[key, 'prediction']])
        #========================================================================
sys.exit()
    
#========================================================================
# Submission
df_pred.set_index(key, inplace=True)
submit[target] = df_pred['prediction']
submit_path = f'../submit/{start_time[4:12]}_submit_{model_type}_set-{set_type}_lgb{len(lgb_cols)}_NN{len(nn_cols)}_other{len(other_cols)}_OUT{str(out_score)[:7]}_CV{cv_score}_LB.csv'
submit.to_csv(submit_path, index=True)
display(submit.head())
#========================================================================

Train: (168264, 20) | Test: (123623, 20)
RMSE: 3.5996719566106026 | SUM ERROR: -201.5692463894596
Train: (168264, 20) | Test: (123623, 20)
RMSE: 3.6266220063391947 | SUM ERROR: 583.8262460140897
Train: (168264, 20) | Test: (123623, 20)
RMSE: 3.631527933137074 | SUM ERROR: 113.61580443130168
Train: (168264, 20) | Test: (123623, 20)
RMSE: 3.6037343997838645 | SUM ERROR: -220.27374647276704
Train: (168264, 20) | Test: (123623, 20)
RMSE: 3.6425257968675515 | SUM ERROR: 41.47717159608197
Train: (168265, 20) | Test: (123623, 20)
RMSE: 3.6063264227887837 | SUM ERROR: -337.630805106333
Stacking Shape: (325540, 3)

#========================================================================
# CV SCORE AVG: 3.618401419254512
# OUT SCORE: 29.72956767421913
#========================================================================


SystemExit: 